In [1]:
import urllib3
import json
import pandas as pd
import zipfile
import requests
import os

In [2]:
def conexao(link):
  http = urllib3.PoolManager()
  request = http.request('GET', link)
  dados = json.loads(request.data.decode('latin1'))
  resultado = dados['result']
  return resultado

def busca_datasets_por_ano(ano):
  pacotes = 'https://dadosabertos.tse.jus.br/api/3/action/package_list'
  resultado = conexao(link=pacotes)
  df = pd.DataFrame(resultado, columns=['Dataset'])
  conjunto = df[df.Dataset.str.contains(str(ano))]
  datasets = conjunto.Dataset.unique().tolist()
  return datasets

def cria_links_com_dataset(dataset):
  return f'https://dadosabertos.tse.jus.br/api/3/action/package_show?id={dataset}'

def filtra_arquivos(link):
  if 'fotos' in link:
    return False
  if 'proposta' in link:
    return False
  if 'convocacao_mesarios' in link:
    return False
  if 'pesquisa_eleitoral' in link:
    return False
  if 'BWEB' in link:
    return False
  if 'corresp' in link:
    return False
  if 'rede_social' in link:
    return False
  return True

def busca_arquivos(datasets):
  arquivos = []
  for dataset in datasets:
    link = cria_links_com_dataset(dataset)
    resultado = conexao(link=link)
    resources = resultado['resources']
    for item in resources:
      url = item['url']
      arquivos.append(url)
  lista_arquivos = filter(filtra_arquivos, arquivos)
  return lista_arquivos

def busca_nome_arquivo(arquivo):
  ultima_barra = arquivo.rfind('/')
  slice_nome = ultima_barra + 1
  return arquivo[slice_nome:]

def cria_dicionario(arquivos):
  dicionario = {}
  for a in arquivos:
    nome = busca_nome_arquivo(a)
    dicionario[nome] = a
  return dicionario

def baixa_arquivos(dicionario, ano):
  for nome, link in dicionario.items():
    resposta = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
    diretorio = os.path.join(os.getcwd(), 'arquivos', str(ano))
    if not os.path.isdir(diretorio):
      os.makedirs(diretorio)
    arquivo = os.path.join(diretorio, nome)
    with open(arquivo, 'wb') as arq:
      arq.write(resposta.content)

In [ ]:
ano = input('De que ano você quer os dados? ')
datasets_ano = busca_datasets_por_ano(ano)
arquivos = busca_arquivos(datasets_ano)
dicionario = cria_dicionario(arquivos)
baixa_arquivos(dicionario, ano)

De que ano você quer os dados? 2022
